In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import shutil
import random
import numpy as np

In [3]:
sys.path.insert(0, '..')
from function import *
from createimages import *

In [4]:
artists = pd.read_csv('../raw_data/artists.csv')
artists.head()

,id,name,years,genre,nationality,bio,wikipedia,paintings
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194


In [5]:
#make id new index and drop id
#artists.index = artists['id']
artists.drop(labels=['id','years','nationality','bio', 'wikipedia'], axis=1, inplace = True)
#rename 'Albrecht Dürer' to 'Albrecht Durer'
artists['name'] = artists['name'].replace(to_replace='Albrecht D.rer', value = 'Albrecht Durer', regex = True)
artists['name'] = artists['name'].str.replace(' ', '_')
#calculate percentage of paintings against total
artists['% of paintings'] = artists.paintings/artists.paintings.sum()*100
artists.head()

,name,genre,paintings,% of paintings
0,Amedeo_Modigliani,Expressionism,193,2.285105
1,Vasiliy_Kandinskiy,"Expressionism,Abstractionism",88,1.041913
2,Diego_Rivera,"Social Realism,Muralism",70,0.828795
3,Claude_Monet,Impressionism,73,0.864314
4,Rene_Magritte,"Surrealism,Impressionism",194,2.296945


In [49]:
artists

,name,genre,paintings,% of paintings
0,Amedeo_Modigliani,Expressionism,193,2.285105
1,Vasiliy_Kandinskiy,"Expressionism,Abstractionism",88,1.041913
2,Diego_Rivera,"Social Realism,Muralism",70,0.828795
3,Claude_Monet,Impressionism,73,0.864314
4,Rene_Magritte,"Surrealism,Impressionism",194,2.296945
5,Salvador_Dali,Surrealism,139,1.645749
6,Edouard_Manet,"Realism,Impressionism",90,1.065593
7,Andrei_Rublev,Byzantine Art,99,1.172152
8,Vincent_van_Gogh,Post-Impressionism,877,10.383614
9,Gustav_Klimt,"Symbolism,Art Nouveau",117,1.385271


In [6]:
#create data frame with artist name, genre, filename, filepath
path = r'..\raw_images'

#get image information
img_info = get_image_info(path)

#convert lists to dataframe
zipped = zip(['name', 'filename', 'filepath'], img_info)
artlabels = pd.DataFrame(dict(list(zipped)))
artlabels.head()

,name,filename,filepath
0,Albrecht_Durer,Albrecht_Durer_(1).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
1,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
2,Albrecht_Durer,Albrecht_Durer_(100).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
3,Albrecht_Durer,Albrecht_Durer_(101).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...
4,Albrecht_Durer,Albrecht_Durer_(102).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...


In [7]:
#extract the artist name and genre
genre = artists.loc[:,['name', 'genre']]

#label each file with their respective genres
artlabels = artlabels.merge(genre, how='left', on='name')
artlabels.head()

,name,filename,filepath,genre
0,Albrecht_Durer,Albrecht_Durer_(1).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
1,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
2,Albrecht_Durer,Albrecht_Durer_(100).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
3,Albrecht_Durer,Albrecht_Durer_(101).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
4,Albrecht_Durer,Albrecht_Durer_(102).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance


In [8]:
#create a new folder for the csv files
datapath = '..\data'
create_new_folder(datapath)

..\data path already exists


In [9]:
#create a new csv file with at most one genre for an artist
#uniqueartlabels = artlabels[~artlabels.genre.str.contains(',')]
#uniqueartlabels.reset_index(drop=True, inplace=True)

newfile = 'artists_clean.csv'
fullpath = os.path.join(datapath, newfile)
artists.to_csv(fullpath)

In [10]:
#create a csv file from the new dataframe
newfile = 'artlabels.csv'
fullpath = os.path.join(datapath, newfile)
artlabels.to_csv(fullpath)
artlabels.head()

,name,filename,filepath,genre
0,Albrecht_Durer,Albrecht_Durer_(1).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
1,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
2,Albrecht_Durer,Albrecht_Durer_(100).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
3,Albrecht_Durer,Albrecht_Durer_(101).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
4,Albrecht_Durer,Albrecht_Durer_(102).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance


In [43]:
def get_random_imgs(oldpath, newpath, df, qty):
    '''
    generate a random sample of files with 
    at most the given qty from each artist
    to each newly created folder
    returns dataframe of files sampled with labels'''
    
    sample_list = [] #pd.DataFrame(columns=['label', 'filename'])
    num_skipped = 0
    copied = 0
    
    if os.path.exists(newpath):
        num_skipped = clear_old_images(newpath, df) 

    for i in range(len(df)):
        rawpath = os.path.join(oldpath, df.name[i])
        files = [f for f in os.listdir(rawpath)]# if os.path.isdir(f)]
        random_files = np.random.choice(files, int(round(qty*df['% of paintings'][i])))                              
        imgpath = os.path.join(newpath, df.genre[i])
        #check if folder exists; if it exists, clear any old residual images before new samples
        if os.path.exists(imgpath):
            pass
        #    num_skipped+=clear_old_images(imgpath)
        else:
            create_new_folder(imgpath)
        #create a dataframe to return label and filename  
        for copy in random_files:
            path_to_copy = os.path.join(rawpath, copy)
            shutil.copy(path_to_copy, imgpath)
            sample_list.append([df.genre[i], copy]) #append(path_to_copy) #df.genre[i] and copy
            copied += 1
    
            
    print("Deleted %d images" % num_skipped)
    print("Generated %d new images" % copied)
    
    return sample_list

In [44]:
def clear_old_images(path, df):
    num_skipped = 0
    #listdir = os.listdir(imagepath)
    #number_files = len(listdir)

    #if number_files >= df.sample_quantity.sum(): #num_folders >= len(artlabels.genre.unique())
    for i in range(len(df)):
        imgpath = os.path.join(path, df.genre[i])
        if os.path.exists(imgpath):
            for fname in os.listdir(imgpath):
                if not fname.startswith('.'):
                    fpath = os.path.join(imgpath, fname)
                    #print(fpath)
                    num_skipped += 1
                    #Delete image
                    os.remove(fpath)
    #print("Deleted %d images" % num_skipped) #from % imagepath

    return num_skipped

In [45]:
#clear old images to procure a new sample
rawpath = r'..\raw_images'
imagepath = r'..\images'
sample_list = get_random_imgs(rawpath, imagepath, artists, 6)

Deleted 0 images
Generated 599 new images


In [61]:
df_sample = pd.DataFrame(columns=['label', 'filename'])
for i in range(len(sample_list)):
    df_sample.loc[i] = sample_list[i]  

In [62]:
df_sample.head()

,label,filename
0,Expressionism,Amedeo_Modigliani_190.jpg
1,Expressionism,Amedeo_Modigliani_185.jpg
2,Expressionism,Amedeo_Modigliani_157.jpg
3,Expressionism,Amedeo_Modigliani_180.jpg
4,Expressionism,Amedeo_Modigliani_5.jpg


In [63]:
df_sample.to_csv(r'..\data\sample.csv')